# Environment Setup


## Import Modules

In [9]:
import numpy as np
import tensorflow as tf
from random import Random
from functools import reduce

## Move Methods

In [10]:
class Move:
    "A collection of valid moves that can be applied to a rubiks cube"
    def build(moves: list[list[int]]) -> np.ndarray[(54,54),np.int8]:
        "Builds a permutation array based on the loop cycles specified in moves"
        m = np.identity(9 * 6, dtype=np.int8)
        for loop in moves:
            first = np.copy(m[loop[0]])
            for i in range(len(loop) - 1):
                m[loop[i]] = m[loop[i + 1]]
            m[loop[-1]] = first
        return m
    def two(moves) -> np.ndarray[(54,54),np.int8]:
        "Builds a permutation array by applying a specified permutation array twice"
        return moves @ moves
    def prime(moves) -> np.ndarray[(54,54),np.int8]:
        "Builds a permutation array by taking the transpose of a specified permutation array"
        return moves.T

    R: np.ndarray[(54,54),np.int8] = build(
        [
            [20, 2, 42, 47],
            [23, 5, 39, 50],
            [26, 8, 36, 53],
            [27, 29, 35, 33],
            [28, 32, 34, 30],
        ]
    )
    R2: np.ndarray[(54,54),np.int8] = two(R)
    RP: np.ndarray[(54,54),np.int8] = prime(R)
    U: np.ndarray[(54,54),np.int8] = build(
        [
            [20, 11, 38, 29],
            [19, 10, 37, 28],
            [18, 9, 36, 27],
            [8, 6, 0, 2],
            [7, 3, 1, 5],
        ]
    )
    U2: np.ndarray[(54,54),np.int8] = two(U)
    UP: np.ndarray[(54,54),np.int8] = prime(U)
    L: np.ndarray[(54,54),np.int8] = build(
        [
            [18, 45, 44, 0],
            [21, 48, 41, 3],
            [24, 51, 38, 6],
            [11, 17, 15, 9],
            [14, 16, 12, 10],
        ]
    )
    L2: np.ndarray[(54,54),np.int8] = two(L)
    LP: np.ndarray[(54,54),np.int8] = prime(L)
    D: np.ndarray[(54,54),np.int8] = build(
        [
            [24, 33, 42, 15],
            [25, 34, 43, 16],
            [26, 35, 44, 17],
            [45, 47, 53, 51],
            [46, 50, 52, 48],
        ]
    )
    D2: np.ndarray[(54,54),np.int8] = two(D)
    DP: np.ndarray[(54,54),np.int8] = prime(D)
    F: np.ndarray[(54,54),np.int8] = build(
        [
            [6, 27, 47, 17],
            [7, 30, 46, 14],
            [8, 33, 45, 11],
            [18, 20, 26, 24],
            [19, 23, 25, 21],
        ]
    )
    FP: np.ndarray[(54,54),np.int8] = prime(F)
    F2: np.ndarray[(54,54),np.int8] = two(F)
    B: np.ndarray[(54,54),np.int8] = build(
        [
            [36, 38, 44, 42],
            [37, 41, 43, 39],
            [29, 0, 15, 53],
            [32, 1, 12, 52],
            [35, 2, 9, 51],
        ]
    )
    BP: np.ndarray[(54,54),np.int8] = prime(B)
    B2: np.ndarray[(54,54),np.int8] = two(B)

MOVES = [Move.R, Move.RP, Move.R2, Move.B, Move.BP, Move.B2, Move.F, Move.FP, Move.F2, Move.D, Move.D2, Move.DP, Move.L, Move.LP, Move.L2, Move.U, Move.U2, Move.UP]
"List of possible moves possible on a rubiks cube"

'List of possible moves possible on a rubiks cube'

## The Cube Environment

In [11]:
def new_cube():
    state = np.zeros((9 * 6), dtype=np.int8)
    for i in range(state.size):
        state[i] = i / 9
    return state

def apply_move(state,move):
    return state @ move

def scramble(state: np.ndarray,count: int):
    random = Random()
    return state @ reduce(lambda a, b: a @ b, [random.choice(MOVES) for i in range(count)])

# Machine Learning Setup


## Converting State to Vector
In order to make an accurate network, we will need to convert the cube's state array to a longer array to make it clearer to the network what color is where

In [12]:
def state_to_vector(state):
    vector = np.zeros((9 * 6 * 6,1),dtype=np.float32)
    for i in range(9 * 6):
        color = state[i]
        vector[i * 6 + color] = 1
    return vector.T
        

## The Neural Network

**For starters:** The neural network will be a 1-layer neural network.

### The network as a variable
In order to store different iterations for machine learning, we will store the network as a tuple with the variables in order from W1, b1, W2, b2, etc.

In [13]:
def random_network(sizes: list[int]) -> list[tuple[(tf.Variable,tf.Variable)]]:
    values = []
    for i in range(len(sizes)):
        size = sizes[i]
        prev_size = 9 * 6 * 6
        if i > 0:
            prev_size = sizes[i-1]
        weights = tf.Variable(tf.random.normal([prev_size, size],stddev=0.03),name=f'W{i+1}')
        constants = tf.Variable(tf.random.normal([size]),name=f'b{i+1}')
        values.append((weights,constants))
    return values


def feed_network(state,network: list[tuple[(tf.Variable,tf.Variable)]]):
    x = tf.cast(state,tf.float32)
    for i in range(len(network)):
        W, b = network[i]
        if i > 0:
            x = tf.nn.relu(x)
        x = tf.add(tf.matmul(x,W),b)
    return x

def copy_network(network: list[tuple[(tf.Variable,tf.Variable)]]):
    copy = []
    for layer in network:
        W,b = layer
        copy.append((np.copy(W),np.copy(b)))
    return copy

In [29]:
network = random_network([20,20,len(MOVES)])

cube = scramble(new_cube(),1000)

out = feed_network(state_to_vector(cube),network)
print(out)

tf.Tensor(
[[ 1.0293078  -0.16887757 -0.6320611   0.4589032   1.7971443  -1.757435
  -0.19497235 -2.302253   -1.0742339  -2.2565904   1.0133637   1.6988068
   0.09972991 -1.0760252   2.3466172  -0.2814953   0.3366325  -0.14630464]], shape=(1, 18), dtype=float32)
